# The Human in the Loop

In the previous chapter, we perfected our knowledge of the standard supervised learning workflows. In this chapter, we will critically examine the ways in which expert knowledge is incorporated in supervised learning. This is done through the identification of the appropriate unit of analysis which might require feature engineering across multiple data sources, through the sometimes imperfect process of labeling examples, and through the specification of a loss function that captures the true business value of errors made by our machine learning model.

In [41]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix

## Data fusion

Originaly, we used the _destination_ computer (from the `attack` dataset) as our entity of interest, and search for the  _destination_ in the `flow` dataset. However, our cybersecurity analyst just told us that it is the infected machines that generate the bad traffic, and will therefore appear as a _source_, not a _destination_, in the `flows` dataset.


In [2]:
def featurize(df):
    return {
        'unique_ports': len(set(df['destination_port'])),
        'average_packet': np.mean(df['packet_count']),
        'average_duration': np.mean(df['duration'])
    }

In [3]:
attack = pd.read_csv('data/redteam.csv')
display(attack.head())
print("#"*40)

flows = pd.read_csv('data/lanl_flows.csv')
display(flows.head())
print("#"*40)

,time,user_domain,source_computer,destination_computer
0,151036,U748@DOM1,C17693,C305
1,151648,U748@DOM1,C17693,C728
2,151993,U6115@DOM1,C17693,C1173
3,153792,U636@DOM1,C17693,C294
4,155219,U748@DOM1,C17693,C5693


########################################


,time,duration,source_computer,source_port,destination_computer,destination_port,protocol,packet_count,byte_count
0,471692,0,C5808,N24128,C26871,N17023,6,1,60
1,471692,0,C5808,N2414,C26871,N19148,6,1,60
2,471692,0,C5808,N24156,C26871,N8001,6,1,60
3,471692,0,C5808,N24161,C26871,N18502,6,1,60
4,471692,0,C5808,N24162,C26871,N11309,6,1,60


########################################


In [4]:
bads = attack.destination_computer.values

# Group by source computer, and apply the feature extractor
out = flows.groupby('source_computer').apply(featurize)

# Convert the iterator to a dataframe by calling list on it
X = pd.DataFrame(list(out), index=out.index)

# Check which sources in X.index are bad to create labels
y = [x in bads for x in X.index]

# Report the average accuracy of Adaboost over 3-fold CV
print(np.mean(cross_val_score(AdaBoostClassifier(), X, y)))

0.9445378151260504


We have successfully incorporated our analyst's feedback. Let's now try to add some more features. 

We will now build on the previous exercise, by considering one additional feature: the number of unique protocols used by each source computer. Note that with grouped data, it is always possible to construct features in this manner: we can take the number of unique elements of all categorical columns, and the mean of all numeric columns as our starting point. 

In [5]:
# Create a feature counting unique protocols per source
protocols = flows.groupby('source_computer').apply(lambda df: len(set(df['protocol'])))

# Convert this feature into a dataframe, naming the column
protocols_DF = pd.DataFrame(protocols, index=protocols.index, columns=['protocol'])

# Now concatenate this feature with the previous dataset, X
X_more = pd.concat([X, protocols_DF], axis=1)

# Refit the classifier and report its accuracy
print(np.mean(cross_val_score(AdaBoostClassifier(), X_more, y)))

0.9495798319327731


In [6]:
X_more

,unique_ports,average_packet,average_duration,protocol
source_computer,,,,
C10,4,222.000000,5.000000,1
C10026,2,21.000000,39.000000,1
C10047,5,21.076923,7.538462,2
C1015,35,5.371429,27.571429,1
C10235,1,11.000000,0.000000,1
...,...,...,...,...
C9873,2,17.000000,4.500000,1
C993,1,5.000000,0.000000,1
C9937,4,7.166667,6.000000,2


We just achieved a further improvement by adding the number of unique protocols used by each source as a feature. 

## Labels, weak labels and truth

We are surprised by the fact that heuristics can be so helpful. So we decide to treat the heuristic that "too many unique ports is suspicious" as a classifier in its own right. We achieve that by thresholding the number of unique ports per source by the average number used in bad source computers -- these are computers for which the label is `True`.

In [7]:
print("Average of unique ports visited by each infected host: {}.".format(np.mean(X[y]['unique_ports'])))
print("Average of unique ports visited per host disregarding labels: {}.".format(np.mean(X['unique_ports'])))

Average of unique ports visited by each infected host: 15.911764705882353.
Average of unique ports visited per host disregarding labels: 11.235294117647058.


In [8]:
446/len(X)

0.7495798319327731

In [9]:
# Split the data into train and test, with 25% as test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Create a new dataset X_train_bad by subselecting bad hosts
X_train_bad = X_train[y_train]

# Calculate the average of unique_ports in bad examples
avg_bad_ports = np.mean(X_train_bad['unique_ports'])

# Label as positive sources that use more ports than that
pred_port = X_test['unique_ports'] > avg_bad_ports

# Print the accuracy of the heuristic
print(accuracy_score(y_test, pred_port))

0.9194630872483222


Isn't it surprising how well a simple heuristic can do on a real problem? Let's see if we can find more such simple rules.

A different cyber analyst tells us that during certain types of attack, the infected source computer sends small bits of traffic, to avoid detection. This makes us wonder whether it would be better to create a combined heuristic that simultaneously looks for large numbers of ports and small packet sizes. Does this improve performance over the simple port heuristic?

In [10]:
# Compute the mean of average_packet for bad sources
avg_bad_packet = np.mean(X_train[y_train]['average_packet'])

# Label as positive if average_packet is lower than that
pred_packet = X_test['average_packet'] < avg_bad_packet

# Find indices where pred_port and pred_packet both True
pred_port = X_test['unique_ports'] > avg_bad_ports
pred_both = pred_packet * pred_port

# Ports only produced an accuracy of 0.919. Is this better?
print(accuracy_score(y_test, pred_both))

0.9328859060402684


The combined rule does pretty well! Often expert knowledge comes in the form of logical combinations of a large number of simple heuristics.

One of our cyber analysts informs you that many of the labels for the first 100 source computers in םור training data might be wrong because of a database error. She hopes we can still use the data because most of the labels are still correct, but asks וד to treat these 100 labels as "noisy". Thankfully we know how to do that, using weighted learning. 

In [15]:
y_train_noisy = y_train
# Fit a Gaussian Naive Bayes classifier to the training data
clf = GaussianNB().fit(X_train, y_train_noisy)

# Report its accuracy on the test data
print(accuracy_score(y_test, clf.predict(X_test)))

# Assign half the weight to the first 100 noisy examples
weights = [0.5]*100 + [1.0]*(len(y_train_noisy)-100)

# Refit using weights and report accuracy. Has it improved?
clf_weights = GaussianNB().fit(X_train, y_train_noisy, sample_weight=weights)
print(accuracy_score(y_test, clf_weights.predict(X_test)))

0.9194630872483222
0.8993288590604027


We now have a way to handle noise in labels, by using weights that reflect our prior beliefs about their accuracy.

## Loss functions Part I

Remember the credit dataset? With all the extra knowledge you now have about metrics, let's have another look at how good a random forest is on this dataset.

In [20]:
credit = pd.read_csv('data/credit.csv')

non_numeric_columns =   ['checking_status',
                         'credit_history',
                         'purpose',
                         'savings_status',
                         'employment',
                         'personal_status',
                         'other_parties',
                         'property_magnitude',
                         'other_payment_plans',
                         'housing',
                         'job',
                         'own_telephone',
                         'foreign_worker']

# Create a label encoder for each column. Encode the values
for column in non_numeric_columns:
    le = LabelEncoder()
    credit[column] = le.fit_transform(credit[column])

credit['class'] = credit['class'] == 'bad'
X, y = credit.drop('class', axis=1), credit['class']

# Split the data into train and test, with 20% as test
X_train, X_test, y_train, y_test = train_test_split(
  X, y, test_size=0.2, random_state=1)

# Create a random forest classifier, fixing the seed to 2
rf_model = RandomForestClassifier(random_state=2).fit(X_train, y_train)

# Use it to predict the labels of the test data
preds = rf_model.predict(X_test)

In [22]:
print(f1_score(y_test, preds))

print(precision_score(y_test, preds))

0.5161290322580646
0.7058823529411765


In [24]:
sum((preds == True) & (y_test == True))

24

In [25]:
# Accuracy is the proportion of examples that were labelled correctly. Compute it without using accuracy_score()
tp = sum((preds == True) & (y_test == True))
tn = sum((preds == False) & (y_test == False))


print((tp + tn)/len(y_test))

0.775


We have mastered a number of performance metrics which will give us a lot of extra options when the time comes to build our own pipeline.

We will still work on the credit dataset for this exercise. Recall that a "positive" in this dataset means "bad credit", i.e., a customer who defaulted on their loan, and a "negative" means a customer who continued to pay without problems. The bank manager informed us that the bank makes 10K profit on average from each "good risk" customer, but loses 150K from each "bad risk" customer. Our algorithm will be used to screen applicants, so those that are labeled as "negative" will be given a loan, and the "positive" ones will be turned down. What is the total cost of our classifier? 

In [28]:
# Fit a random forest classifier to the training data
clf = RandomForestClassifier(random_state=2).fit(X_train, y_train)

# Label the test data
preds = clf.predict(X_test)

# Get false positives/negatives from the confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()

# Now compute the cost using the manager's advice
'''
Falsely classifying a "good" customer as "bad" means that the bank would have lost the chance to make 10K profit. 
Falsely classifying a "bad" customer as "good" means that the bank would have lost 150K due to the customer defaulting on their loan.
'''
cost = fp*10 + fn*150
print(cost)

5350


This sort of analysis is the only way to assess what the actual impact of our classifier will be in the real world.

## Loss functions Part II

We would like to confirm that the `DecisionTreeClassifier()` uses the same default classification threshold as mentioned in the previous lesson, namely 0.5. It seems strange to us that all classifiers should use the same threshold. Let's check!

In [29]:
# Score the test data using the given classifier
scores = clf.predict_proba(X_test)

# Get labels from the scores using the default threshold
preds = [s[1] > 0.5 for s in scores]

# Use the predict method to label the test data again
preds_default = clf.predict(X_test)

# Compare the two sets of predictions
all(preds == preds_default)

True

We have confirmed that this classifier, too, uses 0.5 as a threshold. We will later see how to tune that threshold to fit our purposes.

We heard that the default value of 0.5 maximizes accuracy in theory, but we want to test what happens in practice. So we try out a number of different threshold values, to see what accuracy we get, and hence determine the best-performing threshold value. We repeat this experiment for the F1 score. Is 0.5 the optimal threshold? Is the optimal threshold for accuracy and for the F1 score the same? Go ahead and find out!

In [31]:
# Create a range of equally spaced threshold values
t_range = [0, 0.25, 0.5, 0.75, 1]

# Store the predicted labels for each value of the threshold
preds = [[s[1] > thr for s in scores] for thr in t_range]

# Compute the accuracy for each threshold
accuracies = [accuracy_score(y_test, p) for p in preds]

# Compute the F1 score for each threshold
f1_scores = [f1_score(y_test, p) for p in preds]

# Report the optimal threshold for accuracy, and for F1
print(t_range[np.argmax(accuracies)], t_range[np.argmax(f1_scores)])

0.5 0.25


We were right to be suspicious: in practice, accuracy is sometimes optimized with a threshold other than 0.5. Moreover, if we want to use a different metric, we should re-tune our threshold!

One of the engineers in our arrhythmia detection startup rushes into our office to let us know that there is a problem with the ECG sensor for overweight users. We decide to reduce the influence of examples with weight over 80 by 50%. We are also told that since our startup is targeting the fitness market and makes no medical claims, scaring an athlete unnecessarily is costlier than missing a possible case of arrhythmia. We decide to create a custom loss that makes each "false alarm" ten times costlier than missing a case of arrhythmia. Does down-weighting overweight subjects improve this custom loss?

In [44]:
arrh = pd.read_csv('data/arrh.csv')
# arrh['class'] = arrh['class'] == 'bad'
X, y = arrh.drop('class', axis=1), arrh['class']
# Split the data into train and test, with 20% as test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [45]:
# Create a scorer assigning more cost to false positives
def my_scorer(y_test, y_est, cost_fp=10.0, cost_fn=1.0):
    tn, fp, fn, tp = confusion_matrix(y_test, y_est).ravel()
    return cost_fp*fp + cost_fn*fn

# Fit a DecisionTreeClassifier to the data and compute the loss
clf = DecisionTreeClassifier(random_state=2).fit(X_train, y_train)
print(my_scorer(y_test, clf.predict(X_test)))

122.0


In [46]:
# Create a scorer assigning more cost to false positives
def my_scorer(y_test, y_est, cost_fp=10.0, cost_fn=1.0):
    tn, fp, fn, tp = confusion_matrix(y_test, y_est).ravel()
    return cost_fp*fp + cost_fn*fn

# Fit a DecisionTreeClassifier to the data and compute the loss
clf = DecisionTreeClassifier(random_state=2).fit(X_train, y_train)
print(my_scorer(y_test, clf.predict(X_test)))

# Refit with same seed, downweighting subjects weighing > 80
weights = [0.5 if w > 80 else 1.0 for w in X_train.weight]
clf_weighted = DecisionTreeClassifier(random_state=2).fit(
  X_train, y_train, sample_weight=weights)
print(my_scorer(y_test, clf_weighted.predict(X_test)))

122.0
161.0


We have mastered the art of using weights in order to assign different importance to different parts of the data. Time to revisit our optimization skills using pipelines.